In [1]:
import sys
sys.path.append('../..')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.metrics import binary_classification as bin_metrics
from src.metrics import multiclass_classification as multi_metrics
from src.models.baselines import MulticlassClassifier

In [3]:
# %env CLEARML_WEB_HOST=https://app.clear.ml
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml
# %env CLEARML_API_ACCESS_KEY=VT1X7EH5UU1REKI7T44X
# %env CLEARML_API_SECRET_KEY=NHZDqveny2UDyuTvhVADxwUOAgp2mVv3wyipHnKkCc42hiaJog

In [4]:
# from clearml import Task, Logger

In [5]:
# task = Task.init(
#     project_name='tonality analysis',
#     task_name='TestBaselines',
#     tags=['NaiveBayesClassifier', 'DummyClassifier', 'LogisticRegression'])

In [6]:
path = '../../static/datasets/modified/'
folders = {'bin' : 'bin_classification/', 'multi' : 'multi_classification/'}

In [7]:
#Dummy Classifier
from src.models.baselines import DummyClassifier as Dummy_clf
# task.upload_artifact(name='train_data_preprocessed_for_Dummy', artifact_object='../../../static/datasets/modified/bin_classification/train_data.csv')
# task.upload_artifact(name='val_data_preprocessed_for_Dummy', artifact_object='../../../static/datasets/modified/bin_classification/validation__test.csv')

dummy = Dummy_clf()
train_data = pd.read_csv(path + folders['bin'] + 'train_data.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label
dummy.fit(X_train, y_train)

val_data = pd.read_csv(path + folders['bin'] + 'validation_data.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label

y_pred = dummy.predict(X_val)
print(
    bin_metrics.accuracy_score(y_true=y_val, y_pred=y_pred),
    bin_metrics.recall_score(y_true=y_val, y_pred=y_pred),
    bin_metrics.precision_score(y_true=y_val, y_pred=y_pred),
)


dummy = Dummy_clf()
train_data = pd.read_csv(path + folders['multi'] + 'train_data.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label
dummy.fit(X_train, y_train)

val_data = pd.read_csv(path + folders['multi'] + 'validation_data.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label

y_pred = dummy.predict(X_val)
print(
    multi_metrics.accuracy_score(y_true=y_val, y_pred=y_pred),
    multi_metrics.recall_score(y_true=y_val, y_pred=y_pred),
    multi_metrics.precision_score(y_true=y_val, y_pred=y_pred)
)

0.49 0.44751381215469616 0.43783783783783786
0.6967779730521383 0.5553264604810997 0.5553264604810997


In [ ]:
#Naive Bayes
from src.models.baselines import NaiveBayesClassifier as NB
from sklearn import metrics

nb = NB(mode='cat_features')

train_data = pd.read_csv(path + folders['bin'] + 'bayes_train.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label

nb.fit(X_train, y_train)

val_data = pd.read_csv(path + folders['bin'] + 'bayes_val.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label.values.tolist()

y_pred = nb.predict(X_val)
print(
    multi_metrics.accuracy_score(y_true=y_val, y_pred=y_pred),
    multi_metrics.recall_score(y_true=y_val, y_pred=y_pred, average='macro'),
    multi_metrics.precision_score(y_true=y_val, y_pred=y_pred, average='macro'),
)

nb = NB(mode='cat_features')

train_data = pd.read_csv(path + folders['multi'] + 'bayes_train.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label

nb.fit(X_train, y_train)

val_data = pd.read_csv(path + folders['multi'] + 'bayes_val.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label.values.tolist()

X_val, y_val = X_train, y_train.values.tolist()

y_pred = nb.predict(X_val)
print(
    multi_metrics.accuracy_score(y_true=y_val, y_pred=y_pred),
    multi_metrics.recall_score(y_true=y_val, y_pred=y_pred, average='macro'),
    multi_metrics.precision_score(y_true=y_val, y_pred=y_pred, average='macro'),
)


In [9]:
from sklearn.linear_model import LogisticRegression

#Binary
logreg = LogisticRegression(C=2, penalty='l2', solver='saga')
train_data = pd.read_csv('../../static/embeddings/bin_average_train.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label
logreg.fit(X_train, y_train)

val_data = pd.read_csv('../../static/embeddings/bin_average_val.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label

y_proba = logreg.predict_proba(X_val)
y_pred = logreg.predict(X_val)
print(
    bin_metrics.accuracy_score(y_true=y_val, y_pred=y_pred),
    bin_metrics.recall_score(y_true=y_val, y_pred=y_pred),
    bin_metrics.precision_score(y_true=y_val, y_pred=y_pred),
)

#Multiclass
train_data = pd.read_csv('../../static/embeddings/multi_average_train.csv')
X_train, y_train = train_data.iloc[:, :-1], train_data.label

val_data = pd.read_csv('../../static/embeddings/multi_average_val.csv')
X_val, y_val = val_data.iloc[:, :-1], val_data.label

strategies = ["one-vs-all", "all-vs-all"]
for strategy in strategies:
    multi_clf = MulticlassClassifier(LogisticRegression, strategy)
    multi_clf.fit(X_train, y_train, target_name='label')
    y_pred = multi_clf.predict(X_val)
    print(
        multi_metrics.accuracy_score(y_true=y_val, y_pred=y_pred),
        multi_metrics.recall_score(y_true=y_val, y_pred=y_pred),
        multi_metrics.precision_score(y_true=y_val, y_pred=y_pred)
    )

0.74875 0.6988950276243094 0.7333333333333333
0.7923842999414177 0.6927877947295423 0.6927877947295423
0.7994141769185706 0.7060439560439561 0.7060439560439561


In [ ]:
# task.close()